# この章で学ぶこと
## 説明変数を組み込んだ統計モデル
個体ごとに異なる説明変数によって平均種子数λが変化する統計モデルを考えます。  
このような統計モデル観測データに当てはめることをポアソン回帰（Poisson regression）といい、似たような構造の統計モデルを総称して一般化線形モデル（generalized linear model, GLM）と言います。

# 3.1 個体ごとに平均種子数が異なる例題
ここでは、前章の例と似ている例を扱います。  
大きく異なるのは、個体ごとに肥料を加える処理（施肥処理の有無{T, C}）のラベルがついていることです。

# 3.2 データの外観を調べる

In [2]:
# 必要なライブラリなど読み込み
require 'daru'
require 'rbplotly'
require 'daru/plotly'

include Daru::Plotly::Methods

Object

In [3]:
# データを読み込んで頭の5件を表示
df = Daru::DataFrame.from_csv("data/data3a.csv")
df.head(5)

,y,x,f
0,6,8.31,C
1,6,9.44,C
2,6,9.5,C
3,12,9.07,C
4,10,10.16,C


In [13]:
# y（種子数）だけ頭5件表示
df.y.head(5)

,y
0,6
1,6
2,6
3,12
4,10


In [14]:
# x（個体サイズ）だけ頭5件表示
df.x.head(5)

,x
0,8.31
1,9.44
2,9.5
3,9.07
4,10.16


In [5]:
# f（施肥処理の有無）だけ頭5件表示
#    C: 施肥処理なし
#    T: 施肥処理あり
df.f.head(5)

,f
0,C
1,C
2,C
3,C
4,C


In [6]:
# fの値の要素を数えてみる
#   => "C" と "T" が50個ずつであることが分かる
df.f.value_counts

C,50
T,50


In [19]:
# データフレーム（Daruオブジェクト）の外観をみてみる
df.describe

,y,x
count,100,100
mean,7.83,10.089099999999997
std,2.624880949681338,1.0080485147455993
min,2,7.19
max,15,12.4


## 3.3 図示をする
統計モデリングに取り組むさい、データを色々な図にして眺めるのは非常に重要です。  
Rubyでどのような図示ができるかみていきましょう。

In [7]:
# 散布図を描画する
#Daru::Plotly::Methods.plot(, x: :x, y: :y, type: :scatter).show
traceT = { 
  name: 'T',
  x: df.filter(:row) {|row| row["f"] == "T"}.x.to_a,
  y: df.filter(:row) {|row| row["f"] == "T"}.y.to_a,
  mode: :markers, marker: {color: 'rgba(0, 0, 200, .5)'}
}
traceC = {
  name: 'C',  
  x: df.filter(:row) {|row| row["f"] == "C"}.x.to_a,
  y: df.filter(:row) {|row| row["f"] == "C"}.y.to_a,
  mode: :markers, marker: {color: 'rgba(200, 0, 0, .5)'}
}

Plotly::Plot.new(data: [traceT, traceC]).show

#<Plotly::Offline::HTML:0x007f822244ee08 @id="22293733-f29b-494a-82e6-63205802761d", @data=[{:name=>"T", :y=>[14, 6, 7, 9, 6, 7, 9, 13, 9, 13, 7, 8, 10, 7, 12, 6, 15, 3, 4, 6, 10, 8, 8, 7, 5, 6, 8, 9, 9, 6, 7, 10, 6, 11, 11, 11, 5, 6, 4, 5, 6, 5, 8, 5, 9, 8, 6, 8, 7, 9], :x=>[10.14, 9.05, 9.89, 8.76, 12.04, 9.91, 9.84, 11.87, 10.16, 9.34, 10.17, 10.99, 9.19, 10.67, 10.96, 10.55, 9.69, 10.91, 9.6, 12.37, 10.54, 11.3, 12.4, 10.18, 9.53, 10.24, 11.76, 9.52, 10.4, 9.96, 10.3, 11.54, 9.42, 11.28, 9.73, 10.78, 10.21, 10.51, 10.73, 8.85, 11.2, 9.86, 11.54, 10.03, 11.88, 9.15, 8.52, 10.24, 10.86, 9.97], :mode=>:markers, :marker=>{:color=>"rgba(0, 0, 200, .5)"}}, {:name=>"C", :y=>[6, 6, 6, 12, 10, 4, 9, 9, 9, 11, 6, 10, 6, 10, 11, 8, 3, 8, 5, 5, 4, 11, 5, 10, 6, 6, 7, 9, 3, 10, 2, 9, 10, 5, 11, 10, 4, 8, 9, 12, 8, 9, 8, 6, 6, 10, 10, 9, 12, 6], :x=>[8.31, 9.44, 9.5, 9.07, 10.16, 8.32, 10.61, 10.06, 9.93, 10.43, 10.36, 10.15, 10.92, 8.85, 9.42, 11.11, 8.02, 11.93, 8.55, 7.19, 9.83, 10.79, 8.89, 10.09, 11.63, 10.21, 9.45, 10.44, 9.44, 10.48, 9.43, 10.32, 10.33, 8.5, 9.41, 8.96, 9.67, 10.26, 10.36, 11.8, 10.94, 10.25, 8.74, 10.46, 9.37, 9.74, 8.95, 8.74, 11.32, 9.25], :mode=>:markers, :marker=>{:color=>"rgba(200, 0, 0, .5)"}}], @layout={}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<CZTop::Socket::PUB:0x7f8223580f30 last_endpoint="tcp://127.0.0.1:64263">

In [8]:
# 箱ひげ図
traceT = { 
  name: 'T', 
  y: df.filter(:row) {|row| row["f"] == "T"}.y.to_a,
  type: :box
}
traceC = {
  name: 'C', 
  y: df.filter(:row) {|row| row["f"] == "C"}.y.to_a,
  type: :box
}

Plotly::Plot.new(data: [traceT, traceC]).show

#<Plotly::Offline::HTML:0x007f82228b8188 @id="9fd9a751-bbf4-4851-af4a-48a365bfbebb", @data=[{:name=>"T", :y=>[14, 6, 7, 9, 6, 7, 9, 13, 9, 13, 7, 8, 10, 7, 12, 6, 15, 3, 4, 6, 10, 8, 8, 7, 5, 6, 8, 9, 9, 6, 7, 10, 6, 11, 11, 11, 5, 6, 4, 5, 6, 5, 8, 5, 9, 8, 6, 8, 7, 9], :type=>:box}, {:name=>"C", :y=>[6, 6, 6, 12, 10, 4, 9, 9, 9, 11, 6, 10, 6, 10, 11, 8, 3, 8, 5, 5, 4, 11, 5, 10, 6, 6, 7, 9, 3, 10, 2, 9, 10, 5, 11, 10, 4, 8, 9, 12, 8, 9, 8, 6, 6, 10, 10, 9, 12, 6], :type=>:box}], @layout={}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<CZTop::Socket::PUB:0x7f8223580f30 last_endpoint="tcp://127.0.0.1:64263">

## 3.4 ポアソン回帰の統計モデル
ここでは、平均種子数 ${\lambda_i}$ が個体サイズxや施肥処理fに影響されるモデルを設計していきます。  
説明変数は ${x_i}$ で、応答変数は種子数 ${y_i}$ です。  
施肥効果 ${f_i}$ は上記の図をみる限りあまり影響がなさそうなので、いっったん無視します。  
ある個体 ${i}$ で種子数が ${y_i}$ である確率 ${p(y_i|\lambda_i)}$ はポアソン分布にしてがっているものとします。  

$$ p(y|\lambda)=\frac{\lambda^y exp(-\lambda)}{y!}$$